## Import libraries

In [1]:
import folium
import pandas as pd
import json
from numpy import median
from folium import Choropleth, Circle, Marker, Icon, Map
import ast

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/df_api.csv')

In [3]:
df.head()

,name,total_money_raised,offices_country_code,offices_state_code,offices_latitude,offices_longitude,offices_address_1,offices_address_2,offices_zip_code,vegan_rest,num_vegan_rest,preschool,num_preschool,starbucks,num_starbucks,airport,num_airport,clubs,num_clubs,weighted_punct
0,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",10,"[{'fsq_id': '4d370251e4b4a09380322936', 'categ...",3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98
1,Aviary,$19M,USA,NY,40.749376,-73.996435,243 West 30th Street,11th Floor,10001,"[{'fsq_id': '5f161e6f978e663e91ab7155', 'categ...",10,"[{'fsq_id': '4d370251e4b4a09380322936', 'categ...",2,"[{'fsq_id': '55a555fd498e1727ae82a345', 'categ...",10,"[{'fsq_id': '4adb6119f964a520a72621e3', 'categ...",10,"[{'fsq_id': '4cc0b629f82ebfb7b4af6f91', 'categ...",8,821.40
2,Gilt Groupe,$236M,USA,NY,40.747270,-73.980064,2 Park Ave Fl 4,NaN,10016-5602,"[{'fsq_id': '4b02f734f964a520814b22e3', 'categ...",10,"[{'fsq_id': '52712a6911d25f0e0dfa94cb', 'categ...",1,"[{'fsq_id': '4aa01922f964a520483e20e3', 'categ...",10,"[{'fsq_id': '4a5623d5f964a52081b41fe3', 'categ...",10,"[{'fsq_id': '741d73901e18012f487f54d8', 'categ...",4,732.10
3,99designs,$35M,USA,CA,37.795531,-122.400598,447 Battery St.,3rd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",10,"[{'fsq_id': '558a380a498e13330910aef6', 'categ...",1,"[{'fsq_id': '6167c8968b400716f1d65e0b', 'categ...",7,"[{'fsq_id': '535ece06498e564c67f2f049', 'categ...",9,"[{'fsq_id': '4f32bd4d19836c91c7f53884', 'categ...",7,699.97
4,Minted,$52.7M,USA,CA,37.797435,-122.403175,747 Front Street,2nd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",10,"[{'fsq_id': '5cc0a9cac0c0840025deceea', 'categ...",3,"[{'fsq_id': '49d0da91f964a520395b1fe3', 'categ...",3,"[{'fsq_id': 'eb55470d48024203bb09894b', 'categ...",3,"[{'fsq_id': '55ed1abb498e9e55d2e0edfc', 'categ...",9,607.15


In [4]:
df_subset = df[df['weighted_punct'] > 600]

In [5]:
df_subset['vegan_rest'] = df_subset['vegan_rest'].apply(ast.literal_eval)
df_subset['preschool'] = df_subset['preschool'].apply(ast.literal_eval)
df_subset['starbucks'] = df_subset['starbucks'].apply(ast.literal_eval)
df_subset['airport'] = df_subset['airport'].apply(ast.literal_eval)
df_subset['clubs'] = df_subset['clubs'].apply(ast.literal_eval)

In [6]:
def pre_explode(x, name):
    new_list = []
    for i in df_subset[name][0]:
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]
        new_list.append((lat, lon))
    return new_list

In [7]:
# Vegan restaurant
df_subset['vegan_rest_coord'] = df_subset['vegan_rest'].apply(lambda row: pre_explode(row, 'vegan_rest'))
df_subset = df_subset.explode('vegan_rest_coord')
df_subset = df_subset.reset_index(drop=True)
df_subset[['vegan_rest_lat', 'vegan_rest_lon']] = df_subset['vegan_rest_coord'].apply(lambda x: pd.Series([x[0], x[1]]))

In [8]:
# Preschool preschool
df_subset['preschool_coord'] = df_subset['preschool'].apply(lambda row: pre_explode(row, 'preschool'))
df_subset = df_subset.explode('preschool_coord')
df_subset = df_subset.reset_index(drop=True)
df_subset[['preschool_lat', 'preschool_lon']] = df_subset['preschool_coord'].apply(lambda x: pd.Series([x[0], x[1]]))

In [9]:
# Starbucks starbucks
df_subset['starbucks_coord'] = df_subset['starbucks'].apply(lambda row: pre_explode(row, 'starbucks'))
df_subset = df_subset.explode('starbucks_coord')
df_subset = df_subset.reset_index(drop=True)
df_subset[['starbucks_lat', 'starbucks_lon']] = df_subset['starbucks_coord'].apply(lambda x: pd.Series([x[0], x[1]]))

In [10]:
# Airport airport
df_subset['airport_coord'] = df_subset['airport'].apply(lambda row: pre_explode(row, 'airport'))
df_subset = df_subset.explode('airport_coord')
df_subset = df_subset.reset_index(drop=True)
df_subset[['airport_lat', 'airport_lon']] = df_subset['airport_coord'].apply(lambda x: pd.Series([x[0], x[1]]))

In [11]:
# Clubs clubs
df_subset['clubs_coord'] = df_subset['clubs'].apply(lambda row: pre_explode(row, 'clubs'))
df_subset = df_subset.explode('clubs_coord')
df_subset = df_subset.reset_index(drop=True)
df_subset[['clubs_lat', 'clubs_lon']] = df_subset['clubs_coord'].apply(lambda x: pd.Series([x[0], x[1]]))

In [12]:
df_subset

,name,total_money_raised,offices_country_code,offices_state_code,offices_latitude,offices_longitude,offices_address_1,offices_address_2,offices_zip_code,vegan_rest,...,preschool_lon,starbucks_coord,starbucks_lat,starbucks_lon,airport_coord,airport_lat,airport_lon,clubs_coord,clubs_lat,clubs_lon
0,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.744283, -73.9928)",40.744283,-73.992800
1,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.750002, -73.994816)",40.750002,-73.994816
2,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.748059, -73.994827)",40.748059,-73.994827
3,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.748009, -73.994694)",40.748009,-73.994694
4,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.744068, -73.994757)",40.744068,-73.994757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,Minted,$52.7M,USA,CA,37.797435,-122.403175,747 Front Street,2nd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",...,-73.990906,"(40.750372, -73.993083)",40.750372,-73.993083,"(40.745278, -73.989913)",40.745278,-73.989913,"(40.749299, -73.99885)",40.749299,-73.998850
119996,Minted,$52.7M,USA,CA,37.797435,-122.403175,747 Front Street,2nd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",...,-73.990906,"(40.750372, -73.993083)",40.750372,-73.993083,"(40.745278, -73.989913)",40.745278,-73.989913,"(40.745486, -73.994201)",40.745486,-73.994201
119997,Minted,$52.7M,USA,CA,37.797435,-122.403175,747 Front Street,2nd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",...,-73.990906,"(40.750372, -73.993083)",40.750372,-73.993083,"(40.745278, -73.989913)",40.745278,-73.989913,"(40.74454, -73.993291)",40.744540,-73.993291
119998,Minted,$52.7M,USA,CA,37.797435,-122.403175,747 Front Street,2nd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",...,-73.990906,"(40.750372, -73.993083)",40.750372,-73.993083,"(40.745278, -73.989913)",40.745278,-73.989913,"(40.743474, -73.994777)",40.743474,-73.994777


In [15]:
df_subset = df_subset.drop_duplicates(subset=['vegan_rest_coord', 'preschool_coord', 'starbucks_coord', 'airport_coord', 'clubs_coord'], keep='first')

In [16]:
df_subset

,name,total_money_raised,offices_country_code,offices_state_code,offices_latitude,offices_longitude,offices_address_1,offices_address_2,offices_zip_code,vegan_rest,...,preschool_lon,starbucks_coord,starbucks_lat,starbucks_lon,airport_coord,airport_lat,airport_lon,clubs_coord,clubs_lat,clubs_lon
0,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.744283, -73.9928)",40.744283,-73.992800
1,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.750002, -73.994816)",40.750002,-73.994816
2,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.748059, -73.994827)",40.748059,-73.994827
3,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.748009, -73.994694)",40.748009,-73.994694
4,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.996965,"(40.747304, -73.994797)",40.747304,-73.994797,"(40.750392, -73.994351)",40.750392,-73.994351,"(40.744068, -73.994757)",40.744068,-73.994757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.990906,"(40.750372, -73.993083)",40.750372,-73.993083,"(40.745278, -73.989913)",40.745278,-73.989913,"(40.749299, -73.99885)",40.749299,-73.998850
23996,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.990906,"(40.750372, -73.993083)",40.750372,-73.993083,"(40.745278, -73.989913)",40.745278,-73.989913,"(40.745486, -73.994201)",40.745486,-73.994201
23997,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.990906,"(40.750372, -73.993083)",40.750372,-73.993083,"(40.745278, -73.989913)",40.745278,-73.989913,"(40.74454, -73.993291)",40.744540,-73.993291
23998,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,-73.990906,"(40.750372, -73.993083)",40.750372,-73.993083,"(40.745278, -73.989913)",40.745278,-73.989913,"(40.743474, -73.994777)",40.743474,-73.994777


In [ ]:
df_subset.drop('vegan_rest_coord', axis=1, inplace=True)
df_subset.drop('preschool_coord', axis=1, inplace=True)
df_subset.drop('starbucks_coord', axis=1, inplace=True)
df_subset.drop('airport_coord', axis=1, inplace=True)
df_subset.drop('clubs_coord', axis=1, inplace=True)

In [ ]:
df.drop_duplicates(subset=['col1', 'col2'], keep='first', inplace=True)

In [ ]:
df_subset

In [ ]:
ast.literal_eval(df_subset['vegan_rest'][3])

In [ ]:
p1_lat_NY, p1_lon_NY, p2_lat_NY, p2_lon_NY, p3_lat_NY, p3_lon_NY = df['offices_latitude'][0], df['offices_longitude'][0], df['offices_latitude'][1], df['offices_longitude'][1], df['offices_latitude'][2], df['offices_longitude'][2]

In [ ]:
middle_lat_NY = median([p1_lat_NY, p2_lat_NY, p3_lat_NY])
middle_lon_NY = median([p1_lon_NY, p2_lon_NY, p3_lon_NY])

In [ ]:
testmap_NY = folium.Map(location=[middle_lat_NY, middle_lon_NY], zoom_start=13)

In [ ]:
folium.Circle(location=[p1_lat_NY, p1_lon_NY], popup='Point 1A', fill_color='#ff0000', radius=200, weight=2, color="#000000").add_to(testmap_NY)
folium.Circle(location=[p2_lat_NY, p2_lon_NY], popup='Point 1B', fill_color='#00ff00', radius=200, weight=2, color="#000000").add_to(testmap_NY)
folium.Circle(location=[p3_lat_NY, p3_lon_NY], popup='Point 1C', fill_color='#0000ff', radius=200, weight=2, color="#ffffff").add_to(testmap_NY)

In [ ]:
display(testmap_NY)

In [ ]:
for index, row in df.iterrows():
    
    all_together = {"location": [row["offices_latitude"], row["offices_longitude"]]}
            
    # 1. Check for the conditions of Festividad: to get the diferent Icon
    if row["num_vegan_rest"] != None:
        icon = Icon (
                icon = "laptop",
                prefix = "fa",
                color = "red",
                icon_color = "white")
        
    elif row["num_preschool"] != None:
        icon = Icon (
                icon = "moon-o",
                prefix = "fa",
                color = "white",
                icon_color = "black")
        
    elif row["num_starbucks"] != None:
        icon = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "green",
                icon_color = "white")
        
    elif row["num_airport"] != None:
        icon = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "green",
                icon_color = "white")
        
    elif row["num_clubs"] != None:
        icon = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "green",
                icon_color = "white")
    
    # 2. With the icon: I pass that to the Marker

    new_marker = Marker(**all_together, icon = icon)
    
    # 3. Add a Marker per row
    
    new_marker.add_to(testmap_NY)

testmap_NY

In [ ]:
for index, row in df.iterrows():
    
    all_together = {"location": [row["offices_latitude"], row["offices_longitude"]]}
            
    # 1. Check for the conditions of Festividad: to get the diferent Icon
    if row["num_vegan_rest"] != None:
        icon = Icon (
                icon = "laptop",
                prefix = "fa",
                color = "red",
                icon_color = "white")
      
    # 2. With the icon: I pass that to the Marker

    new_marker = Marker(**all_together, icon = icon)
    
    # 3. Add a Marker per row
    
    new_marker.add_to(testmap_NY)

testmap_NY